In [1]:
import pandas as pd
import numpy as np
import plotly.express as ex
import plotly.graph_objects as go
from scipy.stats import spearmanr

import pulp

from kakarake.parallel_coords import GaussianMixtureclustering

In [2]:
data = pd.read_csv("../c432-88.csv", header=None, sep="\s+")[range(2, 11)]
data = data.rename(columns=dict(zip(range(2,11), range(9))))
distances = spearmanr(data).correlation

In [3]:
distances = - np.abs(distances)

In [4]:
row = col = range(len(distances))

In [5]:
# Min sum variant
prob = pulp.LpProblem("DLAP", pulp.LpMinimize)
x = pulp.LpVariable.dicts("x", (row,col), cat="Binary")
y = pulp.LpVariable.dicts("y", (row), cat="Binary")

In [6]:
prob += pulp.lpSum([distances[r][c]*x[r][c] for r in row for c in col])

In [7]:
prob += pulp.lpSum([y[c] for c in col]) <=3

In [8]:
for r in row:
    prob += pulp.lpSum([x[r][c] for c in col]) == 1

In [9]:
for r in row:
    for c in col:
        prob += x[r][c] <= y[c]

In [10]:
prob.solve()

1

In [11]:
x_val = np.zeros((len(distances),len(distances))) - 100
y_val = np.zeros((len(distances)))
for v in prob.variables():
    if "x" in v.name:
        _, r, c = v.name.split("_")
        x_val[int(r)][int(c)] = v.varValue
    if "y" in v.name:
        _, c = v.name.split("_")
        y_val[int(c)] = v.varValue
print(x_val)
print(y_val)

[[1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]]
[1. 1. 0. 0. 1. 0. 0. 0. 0.]


In [12]:
for id, val in enumerate(y_val):
    if val==1:
        print(f"Cluster centre: {id}, Cluster: {np.where(x_val[:,id])[0]}")

Cluster centre: 0, Cluster: [0 7]
Cluster centre: 1, Cluster: [1 3 5 8]
Cluster centre: 4, Cluster: [2 4 6]


In [13]:
np.where(x_val[:,1])

(array([1, 3, 5, 8], dtype=int64),)

In [14]:
x_val[:,1]

array([0., 1., 0., 1., 0., 1., 0., 0., 1.])

In [15]:
pulp.listSolvers(onlyAvailable=True)

['PULP_CBC_CMD', 'PULP_CHOCO_CMD']

In [106]:
# Min max variant
prob = pulp.LpProblem("DLAP", pulp.LpMinimize)
x = pulp.LpVariable.dicts("x", (row,col), cat="Binary")
y = pulp.LpVariable.dicts("y", (row), cat="Binary")
z = pulp.LpVariable("z")

In [107]:
prob += z

In [108]:
prob += pulp.lpSum([y[c] for c in col]) <=3

In [109]:
for r in row:
    prob += pulp.lpSum([x[r][c] for c in col]) == 1
    prob += z >= pulp.lpSum([x[r][c]*distances[r][c] for c in col])

In [110]:
for r in row:
    for c in col:
        prob += x[r][c] <= y[c]

In [111]:
prob.solve()

1

In [112]:
for id, val in enumerate(y_val):
    if val==1:
        print(f"Group Leader: {id}, Group: [{np.where(x_val[:,id])}]")

Group Leader: 0, Group: [(array([0, 7], dtype=int64),)]
Group Leader: 1, Group: [(array([1, 3, 5, 8], dtype=int64),)]
Group Leader: 4, Group: [(array([2, 4, 6], dtype=int64),)]


In [16]:
#####

In [25]:
sum([distances[r][c]*x[r][c].value() for r in row for c in col])

-8.141549144818052

In [13]:
def solver(distances, Nc):
    row = col = range(len(distances))
    # Min sum variant
    prob = pulp.LpProblem("DLAP", pulp.LpMinimize)
    x = pulp.LpVariable.dicts("x", (row,col), cat="Binary")
    y = pulp.LpVariable.dicts("y", (row), cat="Binary")
    prob += pulp.lpSum([distances[r][c]*x[r][c] for r in row for c in col])
    prob += pulp.lpSum([y[c] for c in col]) <= Nc
    for r in row:
        prob += pulp.lpSum([x[r][c] for c in col]) == 1
    for r in row:
        for c in col:
            prob += x[r][c] <= y[c]
    prob.solve()
    
    x_val = np.zeros((len(distances),len(distances))) - 100
    y_val = np.zeros((len(distances)))
    for v in prob.variables():
        if "x" in v.name:
            _, r, c = v.name.split("_")
            x_val[int(r)][int(c)] = v.varValue
        if "y" in v.name:
            _, c = v.name.split("_")
            y_val[int(c)] = v.varValue
    print(x_val)
    print(y_val)


    groups = []
    for id, val in enumerate(y_val):
        if val==1:
            groups.append(np.where(x_val[:,id])[0])
    objective_val = sum([distances[r][c]*x[r][c].value() for r in row for c in col])
    return len(groups), objective_val

In [18]:
solver(distances, )

[[1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]]
[1. 1. 1. 1. 1. 1. 1. 1. 1.]


(9, -9.0)

In [21]:
Nc_options = list(range(1,10))
objective_vals = [solver(distances, Nc)[1] for Nc in Nc_options]

[[0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]]
[0. 0. 1. 0. 0. 0. 0. 0. 0.]
[[1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]]
[1. 0. 1. 0. 0. 0. 0. 0. 0.]
[[1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]]
[1. 1. 0. 0. 1. 0. 0. 0. 0.]
[[1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0.

In [22]:
objective_vals

[-6.472714426288115,
 -7.588373503540216,
 -8.141549144818052,
 -8.402280680798999,
 -8.628223668248701,
 -8.772256019388106,
 -8.882228626637652,
 -8.949690464151344,
 -9.0]

In [23]:
import matplotlib.pyplot as plt

In [49]:
%matplotlib widget

In [56]:
plt.scatter([x for x in Nc_options], objective_vals)
plt.ylabel(r"$g_1$")
plt.xlabel(r"$g_2 (N_c)$")
plt.title("Optimal Solutions of biobjective variant of DLAP problem")
plt.tight_layout()

In [30]:
[x**2 for x in Nc_options]

[1, 4, 9, 16, 25, 36, 49, 64, 81]